In [7]:
from IPython.display import display
from ml_tools import SimpleAggregator, Hour, SingleAggregator, custom_generator
import pandas as pd

# Transforming features

In [8]:
single_dataset = pd.read_csv("data/ml_tools_single_table_dataset/fraud.csv")
display(single_dataset.sample(10))
## too big to run quickly (also unnecessary for the demo)
single_dataset_trunc = single_dataset.sample(200)

,id,card_id,store_id,datetime,amount,currency,customer_present,provider,lat,lng,region,country,fraud
90186,90186,29950,1100,2019-07-01 16:49:53,69208,KZT,True,Diners Club / Carte Blanche,60.97944,76.92421,Izluchinsk,RU,False
17942,17942,29084,4527,2019-02-25 01:20:29,44160,NPR,False,VISA 19 digit,12.18441,18.69303,Mongo,TD,True
31672,31672,21948,7483,2019-06-06 15:07:25,38709,TOP,True,American Express,-17.74431,-48.62789,Caldas Novas,BR,True
89222,89222,33122,8765,2019-04-07 13:25:02,80495,SRD,False,VISA 19 digit,43.31667,-2.68333,Gernika-Lumo,ES,False
69516,69516,461,1688,2019-03-26 12:04:16,77133,KES,False,JCB 16 digit,25.67927,-80.31727,Kendall,US,False
5223,5223,23154,1055,2019-04-23 01:00:39,31183,HNL,False,VISA 13 digit,51.20219,7.36027,Radevormwald,DE,False
13213,13213,25090,2281,2019-07-07 04:37:24,42490,MVR,False,Mastercard,58.63667,59.80222,Lesnoy,RU,False
93406,93406,8868,2529,2019-02-06 23:51:20,40314,QAR,False,JCB 15 digit,45.44868,-73.81669,Pointe-Claire,CA,False
83815,83815,13392,1768,2019-05-31 16:59:13,63845,BZD,True,VISA 19 digit,53.19146,-2.52398,Winsford,GB,True
97525,97525,20389,9706,2019-04-27 08:06:35,90114,NPR,False,VISA 13 digit,29.84576,-90.10674,Estelle,US,True


In [9]:
single_dataset_trunc.datetime = pd.to_datetime(single_dataset_trunc.datetime)
single_dataset_hours = Hour.generate_feature(single_dataset_trunc.copy())
display(single_dataset_hours.sample(10))

[3]


/Users/john/Projects/Programming/ml_tools/ml_tools/feature_generator.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, idx][index] = time.hour


,id,card_id,store_id,datetime,amount,currency,customer_present,provider,lat,lng,region,country,fraud
68130,68130,32066,997,22,79834,LSL,True,VISA 16 digit,33.52253,-117.70755,Laguna Niguel,US,False
95998,95998,32887,7547,0,97560,HRK,True,Diners Club / Carte Blanche,-26.22861,-52.67056,Pato Branco,BR,True
17279,17279,3550,3550,15,8047,NGN,False,JCB 16 digit,33.37526,74.30920,Rajaori,IN,False
18267,18267,32220,3225,4,58580,GNF,True,VISA 16 digit,34.08565,74.80555,Srinagar,IN,False
44795,44795,28093,785,15,71813,LSL,False,Maestro,36.60090,126.66500,Hongseong,KR,False
3750,3750,2284,8604,7,75604,MRO,True,VISA 19 digit,39.57582,-105.11221,Ken Caryl,US,False
74610,74610,11157,8055,21,52115,XDR,False,VISA 16 digit,52.14000,5.58472,Barneveld,NL,False
36386,36386,16695,5039,3,91532,KHR,False,Mastercard,37.73604,-120.93549,Riverbank,US,True
40959,40959,29730,1288,8,36496,UZS,True,VISA 13 digit,46.18396,6.10237,Onex,CH,False
29828,29828,23835,5330,14,46806,IQD,False,VISA 13 digit,6.25947,102.05461,Tak Bai,TH,False


# Aggregating data
- Across multiple dataframes

In [13]:
transactions = pd.read_csv("data/ml_tools_multiiple_tables_dataset/transactions.csv")
products = pd.read_csv("data/ml_tools_multiiple_tables_dataset/products.csv")

tp_agg = SimpleAggregator(transactions,
                          products,
                          label1='transactions',
                          label2='products')
tp_agg.new_relationship('product_id', 'product_id')
print(tp_agg.relationships)

agg = tp_agg.aggregate()
display(agg)


 RELATIONSHIPS:
transactions.product_id -> products.product_id



,product_id,count
0,4,106
1,5,104
2,1,102
3,3,96
4,2,92


# Aggregating data
- In one dataframe

In [11]:
single_agg = SingleAggregator(single_dataset, 'fraud', 'currency')
currency_agg = single_agg.aggregate()

display(currency_agg)


,currency,count
0,TJS,677
1,VUV,667
2,CAD,667
3,NAD,666
4,CZK,664
...,...,...
159,SLL,557
160,LTL,556
161,PAB,542
162,AUD,541


# Custom feature generators

In [12]:
def square_column(data, column):
    data[column+'_squared'] = data[column] * data[column]
    return data

cust_generator = custom_generator(square_column)

new_data = cust_generator.generate_feature(single_dataset_trunc.copy(), 'amount')

display(new_data)

,id,card_id,store_id,datetime,amount,currency,customer_present,provider,lat,lng,region,country,fraud,amount_squared
83836,83836,12039,5678,2019-07-11 19:33:20,59598,GEL,False,Mastercard,36.20389,127.08472,Nonsan,KR,False,3551921604
46381,46381,29781,7404,2019-01-29 09:34:48,56119,TVD,False,Discover,43.78956,7.60872,Ventimiglia,IT,False,3149342161
27082,27082,22557,3801,2019-05-06 05:01:51,25916,GYD,True,VISA 16 digit,20.28527,-103.42897,Jocotepec,MX,True,671639056
20680,20680,4001,4001,2019-07-14 15:56:58,7112,JEP,False,JCB 15 digit,48.77275,5.16108,Bar-le-Duc,FR,False,50580544
95172,95172,1370,1370,2019-05-10 04:34:27,3123,MRO,False,Discover,5.30383,-1.98956,Tarkwa,GH,False,9753129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95998,95998,32887,7547,2019-07-01 00:10:40,97560,HRK,True,Diners Club / Carte Blanche,-26.22861,-52.67056,Pato Branco,BR,True,9517953600
53644,53644,23649,702,2019-04-27 13:38:24,7242,OMR,False,Diners Club / Carte Blanche,33.03699,-117.29198,Encinitas,US,False,52446564
98902,98902,28224,9106,2019-05-03 12:55:02,42629,CUP,False,VISA 16 digit,45.49428,-122.86705,Aloha,US,False,1817231641
69949,69949,1054,5014,2019-06-04 01:52:05,57470,YER,False,Mastercard,-4.83000,29.65806,Mwandiga,TZ,False,3302800900
